In [40]:
from dotenv import load_dotenv
from azure.identity import DefaultAzureCredential
from azure.core.credentials import AzureKeyCredential
import os
import json

In [41]:

load_dotenv(override=True) # take environment variables from .env.

# Variables not used here do not need to be updated in your .env file
endpoint = os.environ["AZURE_SEARCH_SERVICE_ENDPOINT"]
credential = AzureKeyCredential(os.environ["AZURE_SEARCH_ADMIN_KEY"]) if len(os.environ["AZURE_SEARCH_ADMIN_KEY"]) > 0 else DefaultAzureCredential()
index_name = os.environ["AZURE_SEARCH_INDEX"]
blob_connection_string = os.environ["BLOB_CONNECTION_STRING"]
blob_container_name = os.environ["BLOB_CONTAINER_NAME"]
azure_openai_endpoint = os.environ["AZURE_OPENAI_ENDPOINT"]
azure_openai_key = os.environ["AZURE_OPENAI_KEY"] if len(os.environ["AZURE_OPENAI_KEY"]) > 0 else None
azure_openai_embedding_deployment = os.environ["AZURE_OPENAI_EMBEDDING_DEPLOYMENT"]


index_name_oyd = os.environ["AZURE_SEARCH_INDEX_OYD"]

In [42]:
import azure.search as azsearch
from azure.search.documents.indexes import SearchIndexClient
from azure.search.documents.indexes.models import AnalyzeTextOptions, LexicalAnalyzerName

search_index_client = SearchIndexClient(endpoint, credential=credential)
print(search_index_client.get_index_statistics(index_name))

search_client = search_index_client.get_search_client(index_name_oyd)



{'document_count': 663, 'storage_size': 19386539, 'vector_index_size': 4071272}


In [43]:

search_results = search_client.search(
    search_text="eye",
    include_total_count=True,
    select=['content', 'title', 'id', 'filepath', 'meta_json_string', 'url'] # not including contentVector
    )


print(search_results.get_count())
print(search_results.get_facets())
print(search_results.get_coverage())

1
None
None


In [36]:
for r in search_results: 

    print(json.dumps(r, indent=2))
    print("meta_json_string >> \n", json.dumps(json.loads(r["meta_json_string"]), indent=4))


{
  "content": "Title: Northwind_Health_Plus_Benefits_Details.pdfIn certain cases, Northwind Health may require prior authorization even if the service is not \nlisted above. Your doctor or health care provider should contact Northwind Health to \ndetermine if prior authorization is required prior to providing care.  \nExceptions to Prior Authorization  \nThere are certain services and treatments that are exempt from prior authorizatio n. These \ninclude:  \n\u2022 Routine office visits  \n\u2022 Immunizations  \n\u2022 X-Ray services  \n\u2022 Emergency services  \n\u2022 Family planning services  \n\u2022 Maternity services  \n\u2022 Services and supplies related to diabetes  \n\u2022 Preventive care services  \n\u2022 Mental health and substance abuse services  \n\u2022 Routine eye exams  \n\u2022 Routine dental exams  \nIt is important to note that the list of services and treatments that are exempt from prior \nauthorization is subject to change. Your doctor or health care provide

In [64]:
search_index = search_index_client.get_index(index_name_oyd)

search_index.as_dict()


print('\nFields >>>')
for f in search_index.fields: 
    print(f"{f.name} ({f.type})")


search_index_vector_search = search_index.vector_search

print('\nvector algorithms >>>')
for algo in search_index_vector_search.algorithms: 
    print(f"{algo.name} ")
print('\nvector vectorizers >>>')
for vec in search_index_vector_search.vectorizers: 
    print(f"{vec.name} ")


print(search_index.additional_properties)


Fields >>>
content (Edm.String)
url (Edm.String)
filepath (Edm.String)
title (Edm.String)
meta_json_string (Edm.String)
contentVector (Collection(Edm.Single))
id (Edm.String)

vector algorithms >>>
contentVector_config 

vector vectorizers >>>
{}
